In [28]:
pip install google-api-python-client pandas boto3 Pillow

In [10]:
# youtube API CALL (n times)
from googleapiclient.discovery import build
import pandas as pd
from urllib.request import urlopen
import boto3
import os

# define
max_data_num = 150
youtube = build('youtube', 'v3', developerKey = 'YOUR_KEY')
df = pd.DataFrame(columns = ['video_id', 'thumbnail'])

os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_KEY'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_KEY'
os.environ['AWS_DEFAULT_REGION'] = 'YOUR_KEY'
client = boto3.client('rekognition')
client=boto3.client('rekognition')


[1, 23.0, 0.13716864585876465, 0.3296913206577301, 'CALM', 'Female']


In [13]:
# 날짜 갱신을 위한 함수(들어온 날짜의 바로 이전달 1일 계산)
def calcuate_next_publish_time(time):
    if(time[5:7]=='01'):
        return '%04d-12-01T00:00:00Z'%(int(time[:4])-1)
    else:
        return time[:5]+'%02d-01T00:00:00Z'%(int(time[5:7])-1)

# start_time=2021-06-01T00:00:00Z # 수집 시작할 날짜를 이 형식으로 지정 (수집하는 당일 날짜 혹은 이전에 수집한 데이터 중 가장 예전 값
end_time = '2024-11-01T00:00:00Z'

def get_youtube_vedio_search_list(keyword):
    global end_time
    page_token = ''
    while(len(df)<max_data_num):
        start_time = calcuate_next_publish_time(end_time)
        print(f"\n현재 수집 기간:")
        print(f"publishedBefore: {end_time}")
        print(f"publishedAfter:  {start_time}")

    	## API 호출
        search_response = youtube.search().list(
            type='video',
            maxResults = 50,       #한 번에 가져오는 데이터 수
            part = 'snippet, id',
            q = keyword,		# 검색 키워드
            regionCode = 'KR',		# 지역. 한국은 KR, 미국은 US
            pageToken = page_token,
            publishedBefore = start_time,
            publishedAfter = end_time
        ).execute()
        ## 불러온 데이터 저장
        for item in search_response['items']:
            new_data = {'video_id':'not-yet', 'thumbnail':'not-yet', 'publish_time':'not-yet'}
            new_data['video_id'] = item['id']['videoId']
            new_data['thumbnail'] = item['snippet']['thumbnails']['default']['url']
            new_data['publish_time'] = item['snippet']['publishTime']
            df.loc[len(df)] = new_data

        if ('nextPageToken' in search_response):
            page_token = search_response['nextPageToken']
        else:
            print('count : ', len(df))
            start_time = calcuate_next_publish_time(start_time)
            page_token=''

    ### url open 불가능한 데이터 삭제
    for index in range(len(df)):
        try:
            urlopen(df['thumbnail'][index]).read()
        except:
            df.drop(index, inplace=True)
            print('drop', index)

    df['views']=-1
    df['views'] = df['views'].astype('int64')
    for index in range(len(df)):
        response = youtube.videos().list(
            part = 'statistics',
            id = df.loc[index, 'video_id']
        ).execute()
        if (response['pageInfo']['totalResults'] > 0):
            if('viewCount' in response['items'][0]['statistics']):
                # 문자열을 정수로 변환하여 저장
                df.loc[index, 'views'] = int(response['items'][0]['statistics']['viewCount'])
        else:
            df.loc[index, 'views'] = -2

    ### 조회수 없으면 삭제
    for index in range(len(df)):
        if(df.loc[index, 'views']==-2):
            df.drop(index, inplace=True)

        df.to_csv("./youtube_data.csv", header=True, index=False)

    return df



In [14]:
df = get_youtube_vedio_search_list('ai')
data_original = pd.read_csv("youtube_data.csv")
df = pd.DataFrame(data_original)

In [15]:
# aws rekognition 테스팅
print(df)
import boto3
from urllib.request import urlopen
from PIL import Image

        video_id                                       thumbnail    views
0    2V5qINlecBA  https://i.ytimg.com/vi/2V5qINlecBA/default.jpg    54885
1    8eo4NIwVYW8  https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg     4274
2    LzvRmixcOIU  https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg    21648
3    pWh4u2sXBhU  https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg    38190
4    qyfZeADqbew  https://i.ytimg.com/vi/qyfZeADqbew/default.jpg    29801
..           ...                                             ...      ...
145  TwF78KYGzbM  https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg    43021
146  XKGJTMJVRBs  https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg  4488323
147  Byt6fZZBz_g  https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg    59061
148  DVYEq1qfYcs  https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg  9194841
149  UiMbh_o4utc  https://i.ytimg.com/vi/UiMbh_o4utc/default.jpg   730562

[150 rows x 3 columns]


In [16]:
df['thumbnail_text'] = 'not-yet'
start_index = 0

for index in range(start_index, len(df)):
    try:
        response = client.detect_text(Image={'Bytes': urlopen(df.loc[index, 'thumbnail']).read()})
        df.loc[index, 'thumbnail_text'] = str(response)  # response를 문자열로 변환
    except Exception as e:
        df.loc[index, 'thumbnail_text'] = f'error: {str(e)}'
        print(f"{index} error: {str(e)}")

df.to_csv("youtube_data_with_text.csv", header=True, index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [17]:
import pandas as pd
data_original = pd.read_csv("youtube_data_with_text.csv")
df = pd.DataFrame(data_original)

#content와 area를 저장할 column 추가
df['thumbnail_text_content']='not-yet'
df['thumbnail_text_area']=-1

for index in range(0, len(df)):
    df.at[index, 'thumbnail_text'] = eval(df.at[index, 'thumbnail_text'])
# df 출력
print(df)
# csv 로 저장
df.to_csv("second_youtube_data_with_text_.csv", header=True, index=False)


        video_id                                       thumbnail    views  \
0    2V5qINlecBA  https://i.ytimg.com/vi/2V5qINlecBA/default.jpg    54885   
1    8eo4NIwVYW8  https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg     4274   
2    LzvRmixcOIU  https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg    21648   
3    pWh4u2sXBhU  https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg    38190   
4    qyfZeADqbew  https://i.ytimg.com/vi/qyfZeADqbew/default.jpg    29801   
..           ...                                             ...      ...   
145  TwF78KYGzbM  https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg    43021   
146  XKGJTMJVRBs  https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg  4488323   
147  Byt6fZZBz_g  https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg    59061   
148  DVYEq1qfYcs  https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg  9194841   
149  UiMbh_o4utc  https://i.ytimg.com/vi/UiMbh_o4utc/default.jpg   730562   

                                        thumbnail_text thumbnail_text_conte

In [33]:
import pandas as pd
import ast

# CSV 파일 읽기
data_original = pd.read_csv("second_youtube_data_with_text_.csv")
df = pd.DataFrame(data_original)

# content와 area를 저장할 column 추가 또는 초기화
df['thumbnail_text_content'] = ''
df['thumbnail_text_area'] = 0.0

for index, row in df.iterrows():
    content = ''
    area = 0.0
    try:
        text_data = ast.literal_eval(row['thumbnail_text'])
        for detected in text_data['TextDetections']:
            if detected['Type'] == "LINE" and detected['Confidence'] > 70:
                content += detected['DetectedText'] + ' '
                width = detected['Geometry']['BoundingBox']['Width']
                height = detected['Geometry']['BoundingBox']['Height']
                area += round(width * height, 5)

        df.at[index, 'thumbnail_text_content'] = content.strip()
        df.at[index, 'thumbnail_text_area'] = area
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# 결과 출력
print(df)
# 처리된 데이터를 CSV 파일로 저장
df.to_csv("third_youtube_data_with_text_.csv", index=False)

        video_id                                       thumbnail    views  \
0    2V5qINlecBA  https://i.ytimg.com/vi/2V5qINlecBA/default.jpg    54885   
1    8eo4NIwVYW8  https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg     4274   
2    LzvRmixcOIU  https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg    21648   
3    pWh4u2sXBhU  https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg    38190   
4    qyfZeADqbew  https://i.ytimg.com/vi/qyfZeADqbew/default.jpg    29801   
..           ...                                             ...      ...   
145  TwF78KYGzbM  https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg    43021   
146  XKGJTMJVRBs  https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg  4488323   
147  Byt6fZZBz_g  https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg    59061   
148  DVYEq1qfYcs  https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg  9194841   
149  UiMbh_o4utc  https://i.ytimg.com/vi/UiMbh_o4utc/default.jpg   730562   

                                        thumbnail_text  \
0    {'TextDetect

In [34]:
# print(df)
# 여기서 face detection
for index in range(1, len(df)):
    response = client.detect_faces(
            Image={'Bytes': urlopen(df.loc[index, 'thumbnail']).read()},
            Attributes=['ALL']
        )
    for faceDetail in response['FaceDetails']:
    # 가장 큰 얼굴 하나만 데이터로 저장
        max_height = 0
        height = faceDetail['BoundingBox']['Height']

        if height > max_height:
          max_height = height
        else:
          continue

        # age는 예상 나이 범위의 (low+high)/2
        age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) / 2
        # size
        width = faceDetail['BoundingBox']['Width']
        # height = faceDetail['BoundingBox']['Height']
        # emotion
        emotion_confidence = 0
        emotion = ''

        for item in faceDetail['Emotions']:
            if item['Confidence'] > emotion_confidence:
                emotion_confidence = item['Confidence']
                emotion = item['Type']

        # gender
        gender = faceDetail['Gender']['Value']
        df.loc[index, 'age'] = age
        df.loc[index, 'size'] = width
        df.loc[index, 'emotion'] = emotion
        df.loc[index, 'gender'] = gender

df.to_csv("fourth_youtube_data_with_text_.csv", header=True, index=False)

In [37]:
df = pd.read_csv('fourth_youtube_data_with_text_.csv')
print(df)


        video_id                                       thumbnail    views  \
0    2V5qINlecBA  https://i.ytimg.com/vi/2V5qINlecBA/default.jpg    54885   
1    8eo4NIwVYW8  https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg     4274   
2    LzvRmixcOIU  https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg    21648   
3    pWh4u2sXBhU  https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg    38190   
4    qyfZeADqbew  https://i.ytimg.com/vi/qyfZeADqbew/default.jpg    29801   
..           ...                                             ...      ...   
145  TwF78KYGzbM  https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg    43021   
146  XKGJTMJVRBs  https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg  4488323   
147  Byt6fZZBz_g  https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg    59061   
148  DVYEq1qfYcs  https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg  9194841   
149  UiMbh_o4utc  https://i.ytimg.com/vi/UiMbh_o4utc/default.jpg   730562   

                                        thumbnail_text  \
0    {'TextDetect

In [38]:
# 전처리 진행
data_cat = df.copy()
data_cat['views'] = (data_cat['views'].floordiv(1000))  # 'view'를 'views'로 변경
data_cat['content'] = data_cat['thumbnail_text_content'].fillna('  ')
data_cat['area'] = (data_cat['thumbnail_text_area']*10000).astype('int64')
data_cat.drop(['thumbnail_text_content', 'thumbnail_text_area'], axis='columns', inplace=True)  # 'view' 제거

data_cat

        video_id                                       thumbnail  views  \
0    2V5qINlecBA  https://i.ytimg.com/vi/2V5qINlecBA/default.jpg     54   
1    8eo4NIwVYW8  https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg      4   
2    LzvRmixcOIU  https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg     21   
3    pWh4u2sXBhU  https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg     38   
4    qyfZeADqbew  https://i.ytimg.com/vi/qyfZeADqbew/default.jpg     29   
..           ...                                             ...    ...   
145  TwF78KYGzbM  https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg     43   
146  XKGJTMJVRBs  https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg   4488   
147  Byt6fZZBz_g  https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg     59   
148  DVYEq1qfYcs  https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg   9194   
149  UiMbh_o4utc  https://i.ytimg.com/vi/UiMbh_o4utc/default.jpg    730   

                                        thumbnail_text   age      size  \
0    {'TextDetections': [

In [39]:
# 조회수를 6가지 구간으로 나눈다. 0~5 까지 변환.
import math
for index , view in enumerate(data_cat['views']):
    if view >=10000:  data_cat.loc[index, 'views'] =  5
    elif view >=1000:  data_cat.loc[index, 'views'] =  4
    elif view >=100:  data_cat.loc[index, 'views'] = 3
    elif view >=10: data_cat.loc[index, 'views'] = 2
    elif view > 0: data_cat.loc[index, 'views'] = 1
    else: data_cat.loc[index, 'views'] = 0

data_cat

        video_id                                       thumbnail  views  \
0    2V5qINlecBA  https://i.ytimg.com/vi/2V5qINlecBA/default.jpg      2   
1    8eo4NIwVYW8  https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg      1   
2    LzvRmixcOIU  https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg      2   
3    pWh4u2sXBhU  https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg      2   
4    qyfZeADqbew  https://i.ytimg.com/vi/qyfZeADqbew/default.jpg      2   
..           ...                                             ...    ...   
145  TwF78KYGzbM  https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg      2   
146  XKGJTMJVRBs  https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg      4   
147  Byt6fZZBz_g  https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg      2   
148  DVYEq1qfYcs  https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg      4   
149  UiMbh_o4utc  https://i.ytimg.com/vi/UiMbh_o4utc/default.jpg      3   

                                        thumbnail_text   age      size  \
0    {'TextDetections': [

In [22]:
pip install tensorflow keras

In [40]:
from keras.utils import to_categorical
# one-hot endcoding
y = to_categorical(data_cat['views']).astype('int64')

array([[0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
 

In [41]:
# 4-3-1) 문자열을 token list로 변환
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
data_cat['content_tokens']=None
for index in range(len(df)):
    data_cat['content_tokens'][index] = word_tokenize(data_cat['content'][index])
data_cat

        video_id                                       thumbnail  views  \
0    2V5qINlecBA  https://i.ytimg.com/vi/2V5qINlecBA/default.jpg      2   
1    8eo4NIwVYW8  https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg      1   
2    LzvRmixcOIU  https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg      2   
3    pWh4u2sXBhU  https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg      2   
4    qyfZeADqbew  https://i.ytimg.com/vi/qyfZeADqbew/default.jpg      2   
..           ...                                             ...    ...   
145  TwF78KYGzbM  https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg      2   
146  XKGJTMJVRBs  https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg      4   
147  Byt6fZZBz_g  https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg      2   
148  DVYEq1qfYcs  https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg      4   
149  UiMbh_o4utc  https://i.ytimg.com/vi/UiMbh_o4utc/default.jpg      3   

                                        thumbnail_text   age      size  \
0    {'TextDetections': [

<ipython-input-42-2d491271de48>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_cat['content_tokens'][index] = word_tokenize(data_cat['content'][index])
<ipython-input-42-2d491271de48>:3: SettingWithCopyWarning: 
A value is trying to b

In [43]:
# 원형으로 바꿔주기
from nltk.stem import PorterStemmer
st1 = PorterStemmer()
for index in range(len(df)):
    data_cat['content_tokens'][index] = [st1.stem(w) for w in data_cat['content_tokens'][index]]

data_cat

        video_id                                       thumbnail  views  \
0    2V5qINlecBA  https://i.ytimg.com/vi/2V5qINlecBA/default.jpg      2   
1    8eo4NIwVYW8  https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg      1   
2    LzvRmixcOIU  https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg      2   
3    pWh4u2sXBhU  https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg      2   
4    qyfZeADqbew  https://i.ytimg.com/vi/qyfZeADqbew/default.jpg      2   
..           ...                                             ...    ...   
145  TwF78KYGzbM  https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg      2   
146  XKGJTMJVRBs  https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg      4   
147  Byt6fZZBz_g  https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg      2   
148  DVYEq1qfYcs  https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg      4   
149  UiMbh_o4utc  https://i.ytimg.com/vi/UiMbh_o4utc/default.jpg      3   

                                        thumbnail_text   age      size  \
0    {'TextDetections': [

<ipython-input-43-0121db425fa1>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_cat['content_tokens'][index] = [st1.stem(w) for w in data_cat['content_tokens'][index]]
<ipython-input-43-0121db425fa1>:5: SettingWithCopyWarning: 
A value

In [46]:
data_cat.to_csv("fifth_youtube_data_with_text_.csv", header=True, index=False)

        video_id                                       thumbnail  views  \
0    2V5qINlecBA  https://i.ytimg.com/vi/2V5qINlecBA/default.jpg      2   
1    8eo4NIwVYW8  https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg      1   
2    LzvRmixcOIU  https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg      2   
3    pWh4u2sXBhU  https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg      2   
4    qyfZeADqbew  https://i.ytimg.com/vi/qyfZeADqbew/default.jpg      2   
..           ...                                             ...    ...   
145  TwF78KYGzbM  https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg      2   
146  XKGJTMJVRBs  https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg      4   
147  Byt6fZZBz_g  https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg      2   
148  DVYEq1qfYcs  https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg      4   
149  UiMbh_o4utc  https://i.ytimg.com/vi/UiMbh_o4utc/default.jpg      3   

                                        thumbnail_text   age      size  \
0    {'TextDetections': [

In [47]:
# emotions, gender one-hot encoding
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv('fifth_youtube_data_with_text_.csv')

df = pd.merge(df, pd.get_dummies(df['emotion'], dummy_na=True), left_index=True, right_index=True, how='left')
df = pd.merge(df, pd.get_dummies(df['gender'], dummy_na=True),left_index=True, right_index=True, how='left')
df = df.drop(['emotion', 'gender'], axis='columns')

df

,video_id,thumbnail,views,thumbnail_text,age,size,content,area,content_tokens,ANGRY,CALM,CONFUSED,FEAR,HAPPY,SAD,SURPRISED,nan_x,Female,Male,nan_y
0,2V5qINlecBA,https://i.ytimg.com/vi/2V5qINlecBA/default.jpg,2,{'TextDetections': [{'DetectedText': 'AI NEWS'...,NaN,NaN,AI NEWS,882,"['ai', 'new']",False,False,False,False,False,False,False,True,False,False,True
1,8eo4NIwVYW8,https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg,1,"{'TextDetections': [{'DetectedText': 'BIG AI',...",27.0,0.194199,BIG AI NEWS!,1533,"['big', 'ai', 'new', '!']",False,False,False,False,False,False,True,False,False,True,False
2,LzvRmixcOIU,https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg,2,{'TextDetections': [{'DetectedText': 'BCBS NEW...,27.0,0.079057,BCBS NEWS,343,"['bcb', 'new']",False,True,False,False,False,False,False,False,True,False,False
3,pWh4u2sXBhU,https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg,2,"{'TextDetections': [], 'TextModelVersion': '3....",NaN,NaN,,0,[],False,False,False,False,False,False,False,True,False,False,True
4,qyfZeADqbew,https://i.ytimg.com/vi/qyfZeADqbew/default.jpg,2,{'TextDetections': [{'DetectedText': 'thre - ....,34.0,0.203316,thre - . - - ... - - - - - ... by - G - -,1051,"['thre', '-', '.', '-', '-', '...', '-', '-', ...",False,False,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,TwF78KYGzbM,https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg,2,{'TextDetections': [{'DetectedText': 'NOBEL WI...,NaN,NaN,"NOBEL WINNER: Al POSES ""EXISTENTIAL THREAT"" WA...",1363,"['nobel', 'winner', ':', 'al', 'pose', '``', '...",False,False,False,False,False,False,False,True,False,False,True
146,XKGJTMJVRBs,https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg,4,"{'TextDetections': [], 'TextModelVersion': '3....",37.0,0.112784,,0,[],False,False,False,False,True,False,False,False,False,True,False
147,Byt6fZZBz_g,https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg,2,"{'TextDetections': [{'DetectedText': '-', 'Typ...",26.0,0.271543,- - - - -,242,"['-', '-', '-', '-', '-']",False,False,False,False,False,False,True,False,False,True,False
148,DVYEq1qfYcs,https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg,4,"{'TextDetections': [], 'TextModelVersion': '3....",NaN,NaN,,0,[],False,False,False,False,False,False,False,True,False,False,True


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
def create_combined_features(df):
    # 1. 텍스트 특성 (TF-IDF)
    vectorizer = TfidfVectorizer(
        lowercase=True,
        max_features=1000
    )
    text_features = vectorizer.fit_transform(df['content'].astype(str))

    # 2. 수치형 특성
    numeric_features = df[['age', 'size']].fillna(0)  # NaN값 처리

    # 3. 감정 특성 (이미 원핫 인코딩됨)
    emotion_features = df[['ANGRY', 'CALM', 'CONFUSED', 'FEAR', 'HAPPY', 'SAD', 'SURPRISED']]

    # 4. 성별 특성 (이미 원핫 인코딩됨)
    gender_features = df[['Female', 'Male']]

    # 5. 특성 결합
    X = np.hstack([
        text_features.toarray(),
        numeric_features,
        emotion_features,
        gender_features
    ])

    return X, vectorizer


In [54]:
from sklearn.ensemble import RandomForestRegressor
# 모델 학습
def train_model(df):
    # 특성 생성
    X, vectorizer = create_combined_features(df)
    y = df['views']

    # Random Forest 모델 생성 및 학습
    model = RandomForestRegressor(
        n_estimators=100,
        random_state=42
    )
    model.fit(X, y)

    return model, vectorizer

In [55]:
# 새로운 썸네일에 대한 예측 (실제는 웹에서 할거임)
def predict_views_for_new_thumbnail(model, vectorizer, new_thumbnail):
    """
    new_thumbnail = {
        'content': '새로운 AI 뉴스',
        'age': 30,
        'size': 0.2,
        'emotions': {
            'ANGRY': False,
            'CALM': True,
            'CONFUSED': False,
            'FEAR': False,
            'HAPPY': False,
            'SAD': False,
            'SURPRISED': False
        },
        'gender': {
            'Female': False,
            'Male': True
        }
    }
    """
    # 1. 텍스트 특성
    text_features = vectorizer.transform([new_thumbnail['content']])

    # 2. 수치형 특성
    numeric_features = np.array([[
        new_thumbnail.get('age', 0),
        new_thumbnail.get('size', 0)
    ]])

    # 3. 감정 특성
    emotion_features = np.array([[
        new_thumbnail['emotions'].get('ANGRY', False),
        new_thumbnail['emotions'].get('CALM', False),
        new_thumbnail['emotions'].get('CONFUSED', False),
        new_thumbnail['emotions'].get('FEAR', False),
        new_thumbnail['emotions'].get('HAPPY', False),
        new_thumbnail['emotions'].get('SAD', False),
        new_thumbnail['emotions'].get('SURPRISED', False)
    ]])

    # 4. 성별 특성
    gender_features = np.array([[
        new_thumbnail['gender'].get('Female', False),
        new_thumbnail['gender'].get('Male', False)
    ]])

    # 5. 특성 결합
    X_new = np.hstack([
        text_features.toarray(),
        numeric_features,
        emotion_features,
        gender_features
    ])

    # 예측
    predicted_views = model.predict(X_new)
    return predicted_views[0]

In [56]:

# 사용 예시
# 1. 모델 학습
model, vectorizer = train_model(df)

# 2. 새로운 썸네일에 대한 예측
new_thumbnail = {
    'content': 'NEW AI BREAKTHROUGH!',
    'age': 30,
    'size': 0.2,
    'emotions': {
        'ANGRY': False,
        'CALM': False,
        'CONFUSED': False,
        'FEAR': False,
        'HAPPY': False,
        'SAD': False,
        'SURPRISED': True
    },
    'gender': {
        'Female': False,
        'Male': True
    }
}

predicted_views = predict_views_for_new_thumbnail(model, vectorizer, new_thumbnail)
print(f"예상 조회수: {predicted_views}")

예상 조회수: 2.63


In [57]:
df

,video_id,thumbnail,views,thumbnail_text,age,size,content,area,content_tokens,ANGRY,CALM,CONFUSED,FEAR,HAPPY,SAD,SURPRISED,nan_x,Female,Male,nan_y
0,2V5qINlecBA,https://i.ytimg.com/vi/2V5qINlecBA/default.jpg,2,{'TextDetections': [{'DetectedText': 'AI NEWS'...,NaN,NaN,AI NEWS,882,"['ai', 'new']",False,False,False,False,False,False,False,True,False,False,True
1,8eo4NIwVYW8,https://i.ytimg.com/vi/8eo4NIwVYW8/default.jpg,1,"{'TextDetections': [{'DetectedText': 'BIG AI',...",27.0,0.194199,BIG AI NEWS!,1533,"['big', 'ai', 'new', '!']",False,False,False,False,False,False,True,False,False,True,False
2,LzvRmixcOIU,https://i.ytimg.com/vi/LzvRmixcOIU/default.jpg,2,{'TextDetections': [{'DetectedText': 'BCBS NEW...,27.0,0.079057,BCBS NEWS,343,"['bcb', 'new']",False,True,False,False,False,False,False,False,True,False,False
3,pWh4u2sXBhU,https://i.ytimg.com/vi/pWh4u2sXBhU/default.jpg,2,"{'TextDetections': [], 'TextModelVersion': '3....",NaN,NaN,,0,[],False,False,False,False,False,False,False,True,False,False,True
4,qyfZeADqbew,https://i.ytimg.com/vi/qyfZeADqbew/default.jpg,2,{'TextDetections': [{'DetectedText': 'thre - ....,34.0,0.203316,thre - . - - ... - - - - - ... by - G - -,1051,"['thre', '-', '.', '-', '-', '...', '-', '-', ...",False,False,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,TwF78KYGzbM,https://i.ytimg.com/vi/TwF78KYGzbM/default.jpg,2,{'TextDetections': [{'DetectedText': 'NOBEL WI...,NaN,NaN,"NOBEL WINNER: Al POSES ""EXISTENTIAL THREAT"" WA...",1363,"['nobel', 'winner', ':', 'al', 'pose', '``', '...",False,False,False,False,False,False,False,True,False,False,True
146,XKGJTMJVRBs,https://i.ytimg.com/vi/XKGJTMJVRBs/default.jpg,4,"{'TextDetections': [], 'TextModelVersion': '3....",37.0,0.112784,,0,[],False,False,False,False,True,False,False,False,False,True,False
147,Byt6fZZBz_g,https://i.ytimg.com/vi/Byt6fZZBz_g/default.jpg,2,"{'TextDetections': [{'DetectedText': '-', 'Typ...",26.0,0.271543,- - - - -,242,"['-', '-', '-', '-', '-']",False,False,False,False,False,False,True,False,False,True,False
148,DVYEq1qfYcs,https://i.ytimg.com/vi/DVYEq1qfYcs/default.jpg,4,"{'TextDetections': [], 'TextModelVersion': '3....",NaN,NaN,,0,[],False,False,False,False,False,False,False,True,False,False,True


In [58]:
import joblib
from datetime import datetime

# 현재 시간을 파일명에 포함하여 버전 관리
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

# 모델 저장
model_filename = f'thumbnail_model_{current_time}.joblib'
joblib.dump(model, model_filename)

# TF-IDF Vectorizer도 함께 저장 (텍스트 처리에 사용된 경우)
vectorizer_filename = f'tfidf_vectorizer_{current_time}.joblib'
joblib.dump(vectorizer, vectorizer_filename)

print(f"모델이 저장되었습니다: {model_filename}")
print(f"Vectorizer가 저장되었습니다: {vectorizer_filename}")

# 나중에 모델을 불러올 때 사용하는 코드
# loaded_model = joblib.load(model_filename)
# loaded_vectorizer = joblib.load(vectorizer_filename)

모델이 저장되었습니다: thumbnail_model_20241104_075327.joblib
Vectorizer가 저장되었습니다: tfidf_vectorizer_20241104_075327.joblib
